### Convolutional Neural Network

In [1]:
import tensorflow as tf
import numpy as np

#### Hyper parameter

In [19]:
EPOCHS = 10

#### Define Model

In [20]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.sequence = list()
        self.sequence.append(tf.keras.layers.Conv2D(16, (3,3), padding='same', activation="relu")) # 28x28x1 -> 28x28x16
        self.sequence.append(tf.keras.layers.Conv2D(16, (3,3), padding='same', activation="relu")) # 28x28x16 -> 28x28x16
        self.sequence.append(tf.keras.layers.MaxPool2D((2,2)))# 28x28x16 -> 14x14x16
        self.sequence.append(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation="relu")) # 14x14x16 -> 14x14x32
        self.sequence.append(tf.keras.layers.Conv2D(32, (3,3), padding='same', activation="relu")) # 14x14x32 -> 14x14x32
        self.sequence.append(tf.keras.layers.MaxPool2D((2,2))) # 14x14x32 -> 7x7x32
        self.sequence.append(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation="relu")) # 7x7x32 -> 7x7x64
        self.sequence.append(tf.keras.layers.Conv2D(64, (3,3), padding='same', activation="relu")) # 7x7x64 -> 7x7x64
        self.sequence.append(tf.keras.layers.Flatten()) #1568 (feature vector)
        # for classification
        self.sequence.append(tf.keras.layers.Dense(2048,activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))
        
    def __call__(self, x, training=False, mask=None):
        for layer in self.sequence:
            x = layer(x)
        return x

#### Training loop    

In [26]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

#### algorithm test

In [27]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

#### ready dataset

In [28]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# x_train : (NUM_SAMPLE, 28, 28) => (NUM_SAMPLE, 28, 28, 1) 
x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

#### Set train environment

In [29]:
# Create model
model = ConvNet()

#Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

#### Run train loop

In [32]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss:{}, Test Accuracy:{}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))

Epoch 1, Loss: 0.06333708763122559, Accuracy: 98.06111145019531, Test Loss:0.035716697573661804, Test Accuracy:98.91999816894531
Epoch 2, Loss: 0.054064467549324036, Accuracy: 98.34416198730469, Test Loss:0.03178257495164871, Test Accuracy:99.038330078125
Epoch 3, Loss: 0.04798305034637451, Accuracy: 98.52933502197266, Test Loss:0.02839864231646061, Test Accuracy:99.13700103759766
Epoch 4, Loss: 0.043345991522073746, Accuracy: 98.66944885253906, Test Loss:0.025927642360329628, Test Accuracy:99.209716796875
Epoch 5, Loss: 0.03987307846546173, Accuracy: 98.77285766601562, Test Loss:0.02378920651972294, Test Accuracy:99.27428436279297
Epoch 6, Loss: 0.03698350116610527, Accuracy: 98.86479187011719, Test Loss:0.02358439192175865, Test Accuracy:99.28312683105469
Epoch 7, Loss: 0.034776151180267334, Accuracy: 98.93647766113281, Test Loss:0.022063571959733963, Test Accuracy:99.3281478881836
Epoch 8, Loss: 0.03277158364653587, Accuracy: 99.00350189208984, Test Loss:0.021114913746714592, Test A